In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import scale, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import plot_confusion_matrix
from sklearn.pipeline import make_pipeline

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')
gender_submission = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

X_train = train.drop('Survived', axis=1).copy()
y_train = train['Survived'].copy()
X_test = test.copy()

X_train.Age.fillna(round(X_train.Age.mean()), inplace=True)
X_test.Age.fillna(round(X_test.Age.mean()), inplace=True)
X_test.Fare.fillna(X_test.Fare.mean(), inplace=True)

irrelevant_columns = ['Ticket', 'Cabin', 'Name', 'PassengerId']
categorical_columns = ['Sex', 'Pclass', 'Embarked']
X_train_encoded = pd.get_dummies(X_train.drop(irrelevant_columns, axis=1).copy(), columns=categorical_columns)
X_test_encoded = pd.get_dummies(X_test.drop(irrelevant_columns, axis=1).copy(), columns=categorical_columns)

X_train_scaled = scale(X_train_encoded)
X_test_scaled = scale(X_test_encoded)
np.isnan(X_test_scaled).any()

X_train_scaled.shape

(891, 12)

## Neural Network Prediction

In [3]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(12,)),  # input layer
    keras.layers.Dense(16, activation='tanh'),  # hidden layer (1)
    keras.layers.Dense(16, activation='tanh'),  #hidden layer (2)
    keras.layers.Dense(2, activation='softmax') # output layer (3)
])

for lr in [0.01, 0.0005]:
    opt = keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=opt,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(X_train_scaled, y_train, epochs=20)

pred = model.predict(X_test_scaled)

2022-02-17 13:09:02.023284: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-02-17 13:09:02.200000: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20
28/28 [==============================] - 1s 2ms/step - loss: 0.5003 - accuracy: 0.7621
Epoch 2/20
28/28 [==============================] - 0s 2ms/step - loss: 0.4459 - accuracy: 0.8092
Epoch 3/20
28/28 [==============================] - 0s 2ms/step - loss: 0.4273 - accuracy: 0.8171
Epoch 4/20
28/28 [==============================] - 0s 2ms/step - loss: 0.4197 - accuracy: 0.8103
Epoch 5/20
28/28 [==============================] - 0s 2ms/step - loss: 0.4225 - accuracy: 0.8171
Epoch 6/20
28/28 [==============================] - 0s 3ms/step - loss: 0.4162 - accuracy: 0.8272
Epoch 7/20
28/28 [==============================] - 0s 3ms/step - loss: 0.4084 - accuracy: 0.8316
Epoch 8/20
28/28 [==============================] - 0s 2ms/step - loss: 0.3970 - accuracy: 0.8316
Epoch 9/20
28/28 [==============================] - 0s 2ms/step - loss: 0.3966 - accuracy: 0.8373
Epoch 10/20
28/28 [==============================] - 0s 2ms/step - loss: 0.3991 - accuracy: 0.8227
Epoch 11/20
28/28 [

In [4]:
pred[:5]

array([[0.94097227, 0.05902773],
       [0.56009716, 0.43990287],
       [0.9765019 , 0.02349818],
       [0.88968605, 0.11031395],
       [0.7446529 , 0.2553471 ]], dtype=float32)

In [5]:
pred = pd.Series(pred.argmax(1))
pred

0      0
1      0
2      0
3      0
4      0
      ..
413    0
414    1
415    0
416    0
417    0
Length: 418, dtype: int64

In [6]:
predictions = pd.DataFrame()
predictions['PassengerId'] = pd.Series(X_test.PassengerId)
predictions['Survived'] = pred
predictions.to_csv('submission.csv', index=False)